In [ ]:
import keras
from keras.preprocessing.image import load_img, img_to_array, save_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Colab Notebooks/data/StyleTrans

import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Colab Notebooks/data/StyleTrans/"

!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/data/StyleTrans
/content/drive/My Drive/Colab Notebooks/data/StyleTrans


In [ ]:
# 변환하려는 이미지 경로
target_image_path = './target.jpg'
# 스타일 이미지 경로
style_reference_image_path = './styleGogh.jpg'

# 생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [ ]:
import numpy as np
from keras.applications import vgg19

In [ ]:
def preprocess_image(image_path):
    img = load_img (image_path, target_size= (img_height, img_width))
    img =  img_to_array (img)
    img = np.expand_dims (img, axis = 0)
    img = vgg19.preprocess_input (img)
    return img

def deprecess_image(x):
     # ImageNet의 평균 픽셀 값을 더합니다
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip ( x, 0 ,255).astype ('uint8')
    return x

In [ ]:
from keras import backend as K

In [ ]:
target_image = K.constant (preprocess_image (target_image_path))
style_reference_image = K.constant (preprocess_image (style_reference_image_path))

# 생성된 이미지를 담을 플레이스홀더
combination_image = K.placeholder((1, img_height, img_width, 3))


# 세 개의 이미지를 하나의 배치로 합칩니다
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듭니다.
# 이 모델은 사전 훈련된 ImageNet 가중치를 로드합니다
model = vgg19.VGG19 (input_tensor= input_tensor,
                      weights = 'imagenet',
                      include_top = False)
print ('모델 로드 완료')

80142336/80134624 [==============================] - 1s 0us/step
모델 로드 완료


In [ ]:
#contents loss 콘텐츠 손실 정의

def content_loss ( base, combination):
    return K.sum (K.square(combination - base))

In [ ]:
#스타일 손실입니다. 유틸리티 함수를 사용해 입력 행렬의 그람 행렬을 계산합니다.
# 이 행렬은 원본 특성 행렬의 상관관계를 기록한 행렬

def gram_matrix(x):
    features = K.batch_flatten (K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot (features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix (style)
    C = gram_matrix (combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square( S - C)) / (4. * (channels **2) * (size ** 2))

In [ ]:
#생성된 이미지의 픽셀을 사용해 계산하는 총 변위 손실입니다.
# 이는 생성된 이미지가 공간적인 연속성을 가지도록 도와주며 
#픽셀의 격자 무늬가 과도하게 나타나는 것을 막아줍니다. 
#이를 일종의 규제 항으로 해석

def total_variation_loss(x):
    a = K.square (
        x[:, :img_height -1, :img_width -1, :] - x[:, 1:, :img_width -1, :])
    b = K.square (
        x[:, :img_height -1, :img_width -1, :] - x[:, :img_width -1, 1:, :])
    return K.sum( K.pow( a+b, 1.25))

In [ ]:
# 층 이름과 활성화 텐서를 매핑한 딕셔너리
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# 콘텐츠 손실에 사용할 층
content_layer = 'block5_conv2'
# 스타일 손실에 사용할 층
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# 손실 항목의 가중치 평균에 사용할 가중치
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# 모든 손실 요소를 더해 하나의 스칼라 변수로 손실을 정의합니다
loss = K.variable (0.)
layer_features = outputs_dict [content_layer]
target_image_features = layer_features [ 0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss (target_image_features, combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[ layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss( style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss (combination_image)



ValueError: ignored